<a href="https://colab.research.google.com/github/MrcCap/AnotherTest4PACS/blob/main/SubtaskA/textCleaning_Marco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leggiamo il dataset

In [32]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/EVALITA')
os.getcwd()

dataset = pd.read_csv("subtaskA_train.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
!pip install clean-text
!pip install transformers
!python -m spacy download it_core_news_sm
!pip install emoji

In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
import string
import numpy as np
import spacy
from spacy.lang.it.examples import sentences 
nlp = spacy.load("it_core_news_sm")
from cleantext import clean
import emoji
import difflib
from cleantext import clean

Text Cleaning
---
definiamo la funzione text_cleaning


In [40]:
# Define the cleaning_text function
def cleaning_text(row, patterns_whitespace,patterns):
    text = row['comment_text']
    removed_text = []
    for pattern in patterns:
        # Use re.findall() to find all the removed characters
        removed_text.extend(re.findall(pattern, row['comment_text']))
        # Use re.sub() to remove the pattern from the text
        text = re.sub(pattern, '', text)

    for pattern in patterns_whitespace:
        # Use re.findall() to find all the removed characters
        removed_text.extend(re.findall(pattern, row['comment_text']))
        # Use re.sub() to remove the pattern from the text
        text = re.sub(pattern, ' ', text)
    # Join the removed text into a single string
    removed_text = ''.join(removed_text)
    # Return a Series with the cleaned text and removed text
    return pd.Series([text, removed_text])


def text_cleaning_final(text):
    text = text.lower()
    text = emoji.replace_emoji(text)
    return text

In [8]:
dataset['clean_text']=dataset['comment_text'].apply(text_cleaning)
dataset

,Id,comment_text,conspiratorial,clean_text
0,0,⚡Se non ci fossero soldati non ci sarebbero gu...,0,se non ci fossero soldati non ci sarebbero gue...
1,1,"21/08/21]( [PRE-PRINT]\n\n📄__ ""Shedding of Inf...",1,shedding of infectious sars cov despite vacci...
2,2,PAURA E DELIRIO ALLA CNN: IL MINISTERO DELLA V...,1,paura e delirio alla cnn il ministero della ve...
3,3,L'Aspirina non aumenta la sopravvivenza dei pa...,0,l aspirina non aumenta la sopravvivenza dei pa...
4,4,L'Italia non puo' dare armi lo vieta la Costit...,0,l italia non puo dare armi lo vieta la costitu...
...,...,...,...,...
1837,1837,avvolge la terra spesso 2000 km Oooh mi ricor...,1,avvolge la terra spesso km oooh mi ricorda la ...
1838,1838,Comunque le recensioni erano negative ancora p...,0,comunque le recensioni erano negative ancora p...
1839,1839,Intanto in Kazakistan la gente che ne ha le pa...,1,intanto in kazakistan la gente che ne ha le pa...
1840,1840,Una profezia ad una conferenza del 2015:\nIl p...,1,una profezia ad una conferenza del il professo...


In [42]:
# Define the patterns to be removed
patterns = [r'(\[.*?\])', r'https?:\/\/.*[\r\n]*',r'<br />',r'\d+',r'\n+','[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}']
patterns_whitespace = [r'\<a href', r'[_"\-;%«»()|+&=*%.,!?:#$@\[\]/]',r'\'',r' +']
# Apply the cleaning_text function to each row of the dataframe
dataset[['clean_text', 'removed_text']] = dataset.apply(lambda row: cleaning_text(row, patterns_whitespace,patterns), axis=1)

dataset['clean_text']=dataset['clean_text'].apply(text_cleaning_final)
dataset

,Id,comment_text,conspiratorial,clean_text,removed_text
0,0,⚡Se non ci fossero soldati non ci sarebbero gu...,0,se non ci fossero soldati non ci sarebbero gue...,.
1,1,"21/08/21]( [PRE-PRINT]\n\n📄__ ""Shedding of Inf...",1,shedding of infectious sars cov despite vacc...,[PRE-PRINT][...][la variante][...][...]2108212...
2,2,PAURA E DELIRIO ALLA CNN: IL MINISTERO DELLA V...,1,paura e delirio alla cnn il ministero della ve...,"9550019198420202014\n\n\n\n\n\n\n\n\n\n:"""".,,,..."
3,3,L'Aspirina non aumenta la sopravvivenza dei pa...,0,l aspirina non aumenta la sopravvivenza dei pa...,"19-,'"
4,4,L'Italia non puo' dare armi lo vieta la Costit...,0,l italia non puo dare armi lo vieta la costitu...,..''
...,...,...,...,...,...
1837,1837,avvolge la terra spesso 2000 km Oooh mi ricor...,1,avvolge la terra spesso km oooh mi ricorda la ...,20002000-'
1838,1838,Comunque le recensioni erano negative ancora p...,0,comunque le recensioni erano negative ancora p...,
1839,1839,Intanto in Kazakistan la gente che ne ha le pa...,1,intanto in kazakistan la gente che ne ha le pa...,"\n\n\n:.,.....[ ..."
1840,1840,Una profezia ad una conferenza del 2015:\nIl p...,1,una profezia ad una conferenza del il professo...,"2015\n\n\n\n\n:_-,.,:.........![' ..."


In [37]:
dataset['removed_text'].to_csv('EVALITA_removedText.csv', sep='\n')


La maggior parte del testo tra parentesi rimosso si riferisce nomi di giornali/paper, date e tipologie di vaccini (Moderna/Pfizer/...). Ogni tanto vengono rimosse frasi che sottolineano il contesto sottointeso.

Notiamo che la parola 'SARS-Cov-2' viene spezzata nelle due stringhe SARS CoV (casi del genere possono essere importanti per la challenge B?)